## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [303]:
import numpy as np
import pandas as pd

# Load up store_income_data.csv
store_income_df = pd.read_csv("store_income_data_task.csv")
store_income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   store_name     1000 non-null   object
 2   store_email    413 non-null    object
 3   department     973 non-null    object
 4   income         1000 non-null   object
 5   date_measured  1000 non-null   object
 6   country        965 non-null    object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [304]:
# Handling invalid data
store_income_df.replace(" ", pd.NA, inplace=True)
store_income_df.replace("/", pd.NA, inplace=True)
store_income_df.replace(".", pd.NA, inplace=True)

# Handling null value use mode in "country" column
mode_country = store_income_df["country"].mode()[0]
print(mode_country)
store_income_df["country"].fillna(mode_country, inplace=True)
store_income_df.info()

SA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   store_name     1000 non-null   object
 2   store_email    413 non-null    object
 3   department     973 non-null    object
 4   income         1000 non-null   object
 5   date_measured  1000 non-null   object
 6   country        1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [305]:
# Function show store_income_df's column unique
def show_column_unique(column = "country"):
    countries = store_income_df[column].unique()
    print(f"There are {len(countries)} unique countries")
    print(countries)
    
# Show the unique values in the "country" column
show_column_unique()

There are 73 unique countries
['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' 'SA' 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' 'UK.' 'England/' 'england' ' Britain'
 'united states of america' 'UK/' 'SA/' 'England.' 'UNITED KINGDOM'
 'America.' 'S.A..' 's.a.' ' U.K' ' United States of America' 'Britain '
 'England' ' SA' 'United States of America.' 'United States of America/'
 'United States.' 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. AfricaSouth Africa ' 'S. AfricaSouth Africa.' 'S. Africa

In [306]:
# Convert the column to lowercase
store_income_df["country"] = store_income_df["country"].str.lower()

# Remove trailing white spaces
store_income_df["country"] = store_income_df["country"].str.strip()

# Show unique values
show_column_unique()

There are 33 unique countries
['united states/' 'britain' 'united states' 'britain/' 'united kingdom'
 'u.k.' 'sa' 'u.k/' 'america' 's.a.' 'england' 'uk' 's.a./' 'u.k'
 'america/' 'sa.' 'uk.' 'england/' 'united states of america' 'uk/' 'sa/'
 'england.' 'america.' 's.a..' 'united states of america.'
 'united states of america/' 'united states.' 's. africasouth africa'
 'britain.' 'united kingdom.' 's. africasouth africa/' 'united kingdom/'
 's. africasouth africa.']


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [307]:
import fuzzywuzzy
from fuzzywuzzy import process

# Set seed for reproducibility
np.random.seed(0)

# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    # print(type(rows_with_matches))

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print("All done!")

In [308]:
countries = store_income_df["country"].unique()
matches = fuzzywuzzy.process.extract("england", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('england', 100),
 ('england/', 100),
 ('england.', 100),
 ('united kingdom', 38),
 ('united kingdom.', 38),
 ('united kingdom/', 38),
 ('united states/', 30),
 ('united states', 30),
 ('united states.', 30),
 ('america', 29)]

In [309]:
# key is match_str, value is ratio
match_dic = {"uk": 40,
             "united kingdom": 100,
             "britain": 100,
             "england": 100,
             "united states": 100,
             "united states of america": 100,
             "america": 100,
             "south africa": 75,
             "sa": 40
            }

# Replace matches in "country" column
for match_str, ratio in match_dic.items():
    replace_matches_in_column(df=store_income_df, column="country", string_to_match=match_str, min_ratio=ratio)

# Show unique values
show_column_unique()

All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
There are 9 unique countries
['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' 'england'
 'united states of america' 'south africa']


In [310]:
# Replace same countries
store_income_df.replace('uk', 'united kingdom', inplace=True)
store_income_df.replace('britain', 'united kingdom', inplace=True)
store_income_df.replace('england', 'united kingdom', inplace=True)
store_income_df.replace('united states of america', 'united states', inplace=True)
store_income_df.replace('america', 'united states', inplace=True)
store_income_df.replace('sa', 'south africa', inplace=True)

# Show unique values
show_column_unique()

There are 3 unique countries
['united states' 'united kingdom' 'south africa']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [383]:
import datetime
from datetime import date

store_income_df["date_measured"] = pd.to_datetime(store_income_df["date_measured"], format="%d-%m-%Y")
store_income_df["days_ago"] = datetime.date.today() - store_income_df["date_measured"].dt.date
store_income_df["days_ago"].tail(10)

990     4455 days, 0:00:00
991     8414 days, 0:00:00
992     5343 days, 0:00:00
993     9197 days, 0:00:00
994     6944 days, 0:00:00
995     6776 days, 0:00:00
996    12182 days, 0:00:00
997     5480 days, 0:00:00
998     4852 days, 0:00:00
999     4530 days, 0:00:00
Name: days_ago, dtype: object